In [ ]:
!pip install -q --upgrade google-generativeai langchain langchain-community langchain-google-genai chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:

In [ ]:
from IPython.display import display, Markdown
import textwrap

def to_markdown(text):
  text = text.replace('*', ' *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings




warnings.filterwarnings("ignore")

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                                google_api_key=GOOGLE_API_KEY,
                               temperature=0.2,
                               convert_system_message_to_human=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
def load__pdf():
    pdf_folder_path = "/content/drive/MyDrive/final/UPDATE"
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    return documents

documents = load__pdf()

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=5000)
context = "\n\n".join(str(p.page_content) for p in documents)
text = text_splitter.split_text(context)

In [ ]:
vector_index = Chroma.from_texts(text, embeddings).as_retriever(search_kwargs={"k":10})

In [ ]:
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

retriever = vector_index
llm = model

# Contextualize question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question
qa_system_prompt = (
    """
    Kamu adalah JOKO, seorang analis hukum yang membantu menjawab pertanyaan terkait hukum.
    Gunakan konteks yang ditemukan untuk menjawab pertanyaan.
    Jika tidak tahu jawabannya, katakan minta maaf dan katakan bahwa pertanyaan belum dapat dijawab dengan dataset saat ini
    Format jawaban wajib dalam bentuk format list of object, seperti contoh berikut:
    {{"full_answer": "Rangkuman jawaban secara keseluruhan",
    '"laws": [{{"jawaban": "pasal ini menyebutkan bahwa...", "undang-undang": "Undang-Undang Nomor ...","BAB": "BAB ..." "pasal": "Pasal ...", "ayat": ayat ...}},
    {{"jawaban": "pasal ini menyebutkan bahwa...", "undang-undang": "Undang-Undang Nomor ...","BAB": "BAB ..." "pasal": "Pasal ...", "ayat": ayat ...}},
    {{"jawaban": "pasal ini menyebutkan bahwa...", "undang-undang": "Undang-Undang Nomor ...","BAB": "BAB ..." "pasal": "Pasal ...", "ayat": ayat ...}}, dan seterusnya]}}'
    sesuaikan output ke dalam format yang sudah dicontohkan.
    Meskipun jawaban anda tidak memiliki undang-undang, tetap gunakan format yang di contohkah. isi jawabanmu di full_answer dan laws di null kan
    {context}
    """
)


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt), MessagesPlaceholder("chat_history"), ("human", "{input}"),
    ]
)

#use create_stuff_documents_chain to feed all retrieved context

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(
history_aware_retriever, question_answer_chain
)

# Usage:
chat_history = []
# Collect chat history here (a sequence of messages)

In [ ]:
query="apa hukuman bagi seseorang yang mengendarai motor tetapi tidak memakai helm?"
qa = rag_chain.invoke({"input": query, "chat_history": chat_history})
chat_history.extend([query, qa["answer"]])

In [ ]:
to_markdown(qa['answer'])

> ```json
> {"full_answer": "Berdasarkan Undang-Undang Nomor 22 Tahun 2009 tentang Lalu Lintas dan Angkutan Jalan, seseorang yang mengendarai sepeda motor tetapi tidak mengenakan helm standar nasional Indonesia sebagaimana dimaksud dalam Pasal 106 ayat (8) dipidana dengan pidana kurungan paling lama 1 (satu) bulan atau denda paling banyak Rp250.000,00 (dua ratus lima puluh ribu rupiah).", "laws": [{"jawaban": "Setiap orang yang mengemudikan Sepeda Motor dan Penumpang Sepeda Motor wajib mengenakan helm yang memenuhi standar nasional Indonesia sebagaimana dimaksud dalam Pasal 106 ayat (8) dipidana dengan pidana kurungan paling lama 1 (satu) bulan atau denda paling banyak Rp250.000,00 (dua ratus lima puluh ribu rupiah).", "undang-undang": "Undang-Undang Nomor 22 Tahun 2009 tentang Lalu Lintas dan Angkutan Jalan", "BAB": "BAB IX LALU LINTAS", "pasal": "Pasal 106", "ayat": "ayat 8"}]}
> ```

In [ ]:
query="bagaimana apabila hanya penumpangnya yang tidak memakai helm?"
qa = rag_chain.invoke({"input": query, "chat_history": chat_history})
to_markdown(qa['answer'])

> ```json
> {"full_answer": "Berdasarkan Undang-Undang Nomor 22 Tahun 2009 tentang Lalu Lintas dan Angkutan Jalan, seseorang yang mengemudikan Sepeda Motor yang membiarkan penumpangnya tidak mengenakan helm sebagaimana dimaksud dalam Pasal 106 ayat (8) dipidana dengan pidana kurungan paling lama 1 (satu) bulan atau denda paling banyak Rp250.000,00 (dua ratus lima puluh ribu rupiah).", "laws": [{"jawaban": "Setiap orang yang mengemudikan Sepeda Motor yang membiarkan penumpangnya tidak mengenakan helm sebagaimana dimaksud dalam Pasal 106 ayat (8) dipidana dengan pidana kurungan paling lama 1 (satu) bulan atau denda paling banyak Rp250.000,00 (dua ratus lima puluh ribu rupiah).", "undang-undang": "Undang-Undang Nomor 22 Tahun 2009 tentang Lalu Lintas dan Angkutan Jalan", "BAB": "BAB IX LALU LINTAS", "pasal": "Pasal 106", "ayat": "ayat 8"}]}
> ```

In [ ]:
chat_history.extend([query, qa["answer"]])

In [ ]:
query="sebutkan tugas tugas seorang presiden"
qa = rag_chain.invoke({"input": query, "chat_history": chat_history})
to_markdown(qa['answer'])

> ```json
> {"full_answer": "Tugas-tugas seorang Presiden Republik Indonesia tercantum dalam Undang-Undang Dasar 1945, terutama dalam Bab III Kekuasaan Pemerintahan Negara. Berikut beberapa tugas utama seorang Presiden:\n\n1.  * *Memegang Kekuasaan Pemerintahan: * * Presiden memegang kekuasaan pemerintahan tertinggi di Indonesia, menjalankan pemerintahan berdasarkan UUD 1945. (Pasal 4 ayat 1)\n2.  * *Dibantu Wakil Presiden: * * Presiden dibantu oleh Wakil Presiden dalam menjalankan tugasnya. (Pasal 4 ayat 2)\n3.  * *Mengajukan Rancangan Undang-Undang: * * Presiden berhak mengajukan rancangan undang-undang kepada Dewan Perwakilan Rakyat (DPR). (Pasal 5 ayat 1)\n4.  * *Menetapkan Peraturan Pemerintah: * * Presiden menetapkan peraturan pemerintah untuk menjalankan undang-undang sebagaimana mestinya. (Pasal 5 ayat 2)\n5.  * *Menetapkan Keadaan Bahaya: * * Presiden menyatakan keadaan bahaya, dengan syarat dan akibat yang ditetapkan dalam undang-undang. (Pasal 12)\n6.  * *Menyatakan Perang, Perdamaian, dan Perjanjian: * * Presiden menyatakan perang, membuat perdamaian, dan perjanjian dengan negara lain dengan persetujuan DPR. (Pasal 11 ayat 1)\n7.  * *Membuat Perjanjian Internasional: * * Presiden membuat perjanjian internasional lainnya dengan persetujuan DPR jika menimbulkan akibat luas dan mendasar bagi rakyat, terkait beban keuangan negara, atau mengharuskan perubahan/pembentukan undang-undang. (Pasal 11 ayat 2)\n8.  * *Mengangkat Duta dan Konsul: * * Presiden mengangkat duta dan konsul dengan memperhatikan pertimbangan DPR. (Pasal 13 ayat 1 dan 2)\n9.  * *Menerima Duta Negara Lain: * * Presiden menerima penempatan duta negara lain dengan memperhatikan pertimbangan DPR. (Pasal 13 ayat 3)\n10.  * *Memberi Grasi dan Rehabilitasi: * * Presiden memberi grasi dan rehabilitasi dengan memperhatikan pertimbangan Mahkamah Agung. (Pasal 14 ayat 1)\n11.  * *Memberi Amnesti dan Abolisi: * * Presiden memberi amnesti dan abolisi dengan memperhatikan pertimbangan DPR. (Pasal 14 ayat 2)\n12.  * *Memberi Gelar, Tanda Jasa, dan Tanda Kehormatan: * * Presiden memberi gelar, tanda jasa, dan tanda kehormatan lainnya yang diatur dalam undang-undang. (Pasal 15)\n13.  * *Membentuk Dewan Pertimbangan: * * Presiden membentuk Dewan Pertimbangan yang bertugas memberikan nasihat dan pertimbangan kepada Presiden, diatur lebih lanjut dalam undang-undang. (Pasal 16)\n14.  * *Mengangkat dan Memberhentikan Menteri: * * Presiden mengangkat dan memberhentikan menteri-menteri negara. (Pasal 17 ayat 2)\n15.  * *Memimpin Sidang Kabinet: * * Presiden memimpin sidang kabinet untuk membahas dan memutuskan kebijakan pemerintahan. (Pasal 17 ayat 3)\n16.  * *Memegang Kekuasaan Tertinggi Atas TNI: * * Presiden memegang kekuasaan tertinggi atas Tentara Nasional Indonesia (TNI). (Pasal 10)\n17.  * *Melantik Presiden dan/atau Wakil Presiden: * * Presiden dan/atau Wakil Presiden dilantik oleh Majelis Permusyawaratan Rakyat (MPR). (Pasal 3 ayat 2)\n18.  * *Memberhentikan Presiden dan/atau Wakil Presiden: * * Presiden dan/atau Wakil Presiden dapat diberhentikan dalam masa jabatannya oleh MPR atas usul DPR. (Pasal 3 ayat 3)\n\nSelain tugas-tugas tersebut, Presiden juga memiliki peran penting dalam menjaga stabilitas politik dan keamanan negara, serta mendorong pembangunan nasional.", "laws": [{"jawaban": "Presiden memegang kekuasaan pemerintahan tertinggi di Indonesia, menjalankan pemerintahan berdasarkan UUD 1945.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 4", "ayat": "ayat 1"}, {"jawaban": "Presiden dibantu oleh Wakil Presiden dalam menjalankan tugasnya.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 4", "ayat": "ayat 2"}, {"jawaban": "Presiden berhak mengajukan rancangan undang-undang kepada Dewan Perwakilan Rakyat (DPR).", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 5", "ayat": "ayat 1"}, {"jawaban": "Presiden menetapkan peraturan pemerintah untuk menjalankan undang-undang sebagaimana mestinya.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 5", "ayat": "ayat 2"}, {"jawaban": "Presiden menyatakan keadaan bahaya, dengan syarat dan akibat yang ditetapkan dalam undang-undang.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 12", "ayat": ""}, {"jawaban": "Presiden menyatakan perang, membuat perdamaian, dan perjanjian dengan negara lain dengan persetujuan DPR.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 11", "ayat": "ayat 1"}, {"jawaban": "Presiden membuat perjanjian internasional lainnya dengan persetujuan DPR jika menimbulkan akibat luas dan mendasar bagi rakyat, terkait beban keuangan negara, atau mengharuskan perubahan/pembentukan undang-undang.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 11", "ayat": "ayat 2"}, {"jawaban": "Presiden mengangkat duta dan konsul dengan memperhatikan pertimbangan DPR.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 13", "ayat": "ayat 1 dan 2"}, {"jawaban": "Presiden menerima penempatan duta negara lain dengan memperhatikan pertimbangan DPR.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 13", "ayat": "ayat 3"}, {"jawaban": "Presiden memberi grasi dan rehabilitasi dengan memperhatikan pertimbangan Mahkamah Agung.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 14", "ayat": "ayat 1"}, {"jawaban": "Presiden memberi amnesti dan abolisi dengan memperhatikan pertimbangan DPR.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 14", "ayat": "ayat 2"}, {"jawaban": "Presiden memberi gelar, tanda jasa, dan tanda kehormatan lainnya yang diatur dalam undang-undang.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 15", "ayat": ""}, {"jawaban": "Presiden membentuk Dewan Pertimbangan yang bertugas memberikan nasihat dan pertimbangan kepada Presiden, diatur lebih lanjut dalam undang-undang.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 16", "ayat": ""}, {"jawaban": "Presiden mengangkat dan memberhentikan menteri-menteri negara.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB V KEMENTERIAN NEGARA", "pasal": "Pasal 17", "ayat": "ayat 2"}, {"jawaban": "Presiden memimpin sidang kabinet untuk membahas dan memutuskan kebijakan pemerintahan.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB V KEMENTERIAN NEGARA", "pasal": "Pasal 17", "ayat": "ayat 3"}, {"jawaban": "Presiden memegang kekuasaan tertinggi atas Tentara Nasional Indonesia (TNI).", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB III KEKUASAAN PEMERINTAHAN NEGARA", "pasal": "Pasal 10", "ayat": ""}, {"jawaban": "Presiden dan/atau Wakil Presiden dilantik oleh Majelis Permusyawaratan Rakyat (MPR).", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB II MAJ ELIS PERMUSYAWARATAN RAKYAT", "pasal": "Pasal 3", "ayat": "ayat 2"}, {"jawaban": "Presiden dan/atau Wakil Presiden dapat diberhentikan dalam masa jabatannya oleh MPR atas usul DPR.", "undang-undang": "Undang-Undang Dasar 1945", "BAB": "BAB II MAJ ELIS PERMUSYAWARATAN RAKYAT", "pasal": "Pasal 3", "ayat": "ayat 3"}]}
> ```

In [ ]:
chat_history.extend([query, qa["answer"]])

In [ ]:
query="bagaimana peran dewan perwakilan rakyat di dalam pemerintahan"
qa = rag_chain.invoke({"input": query, "chat_history": chat_history})
to_markdown(qa['answer'])

> ```json
> {"full_answer": "Dewan Perwakilan Rakyat (DPR) memiliki peran penting dalam pemerintahan, yaitu sebagai lembaga legislatif yang memiliki fungsi legislasi, anggaran, dan pengawasan. \n\nFungsi legislasi DPR adalah membentuk undang-undang. DPR bersama Presiden membahas dan menyetujui rancangan undang-undang yang diajukan oleh Presiden atau anggota DPR. \n\nFungsi anggaran DPR adalah menyetujui anggaran pendapatan dan belanja negara (APBN) yang diajukan oleh Presiden. DPR juga memiliki kewenangan untuk mengawasi pelaksanaan APBN. \n\nFungsi pengawasan DPR adalah mengawasi pelaksanaan undang-undang dan kebijakan pemerintah. DPR memiliki hak interpelasi, hak angket, dan hak menyatakan pendapat untuk menjalankan fungsi pengawasan ini. \n\nSelain itu, DPR juga memiliki peran dalam pemilihan umum, yaitu memilih anggota Dewan Perwakilan Daerah, Presiden dan Wakil Presiden, dan Dewan Perwakilan Rakyat Daerah. DPR juga memiliki peran dalam mengangkat hakim agung dan anggota Badan Pemeriksa Keuangan.", "laws": []}
> ```

In [ ]:
qa['answer']

'Dewan Perwakilan Rakyat (DPR) memiliki peran yang sangat penting dalam sistem pemerintahan di Indonesia. Peran DPR dapat dibagi menjadi beberapa aspek:\n\n**1. Legislatif (Pembuat Undang-Undang):**\n\n* **Membuat dan Menetapkan Undang-Undang:** DPR memiliki kewenangan untuk mengajukan, membahas, dan menetapkan undang-undang bersama pemerintah. \n* **Menetapkan Anggaran:** DPR bersama pemerintah menetapkan Anggaran Pendapatan dan Belanja Negara (APBN) yang mengatur pengeluaran dan pendapatan negara.\n* **Menetapkan Peraturan Perundang-undangan:** DPR dapat membuat peraturan perundang-undangan di bawah undang-undang, seperti Peraturan DPR.\n\n**2. Pengawasan:**\n\n* **Melakukan Pengawasan terhadap Pemerintah:** DPR bertugas mengawasi pelaksanaan tugas pemerintah, baik di tingkat pusat maupun daerah. \n* **Menyusun dan Menetapkan Anggaran:** DPR memiliki hak untuk mengawasi penggunaan anggaran negara yang telah ditetapkan.\n* **Menyusun dan Menetapkan Peraturan Perundang-undangan:** DPR 

In [ ]:
chat_history

['apa hukuman bagi seseorang yang mengendarai motor tetapi tidak memakai helm?',
 '```json\n{"full_answer": "Berdasarkan Undang-Undang Nomor 22 Tahun 2009 tentang Lalu Lintas dan Angkutan Jalan, seseorang yang mengendarai sepeda motor tetapi tidak memakai helm standar nasional Indonesia sebagaimana dimaksud dalam Pasal 106 ayat (8) dipidana dengan pidana kurungan paling lama 1 (satu) bulan atau denda paling banyak Rp250.000,00 (dua ratus lima puluh ribu rupiah).", "laws": [{"jawaban": "Setiap orang yang mengemudikan Sepeda Motor dan Penumpang Sepeda Motor wajib mengenakan helm yang memenuhi standar nasional Indonesia sebagaimana dimaksud dalam Pasal 106 ayat (8) dipidana dengan pidana kurungan paling lama 1 (satu) bulan atau denda paling banyak Rp250.000,00 (dua ratus lima puluh ribu rupiah).", "undang-undang": "Undang-Undang Nomor 22 Tahun 2009 tentang Lalu Lintas dan Angkutan Jalan", "BAB": "BAB IX LALU LINTAS", "pasal": "Pasal 106", "ayat": "ayat (8)"}, {"jawaban": "Setiap orang yan

In [ ]:
query="apa pertanyaan saya sebelumnya"
qa = rag_chain.invoke({"input": query, "chat_history": chat_history})
to_markdown(qa['answer'])

> Pertanyaan Anda sebelumnya adalah:
> 
> "bagaimana peran dewan perwakilan rakyat di dalam pemerintahan" 


In [ ]:
query="bagaimana peran dewan perwakilan rakyat di dalam pemerintahan"
qa = rag_chain.invoke({"input": query, "chat_history": chat_history})
to_markdown(qa['answer'])

> Dewan Perwakilan Rakyat (DPR) memiliki peran yang sangat penting dalam sistem pemerintahan di Indonesia.  Peran DPR dapat dibagi menjadi beberapa aspek:
> 
>  * *1. Legislatif (Pembuat Undang-Undang): * *
> 
>  *  * *Membuat dan Menetapkan Undang-Undang: * * DPR memiliki wewenang untuk mengajukan, membahas, dan menetapkan undang-undang bersama pemerintah. 
>  *  * *Menetapkan Anggaran: * * DPR bersama pemerintah menetapkan Anggaran Pendapatan dan Belanja Negara (APBN) yang mengatur pengeluaran dan pendapatan negara.
>  *  * *Mengawasi Pelaksanaan Undang-Undang: * * DPR memiliki wewenang untuk mengawasi pelaksanaan undang-undang yang telah ditetapkan.
> 
>  * *2. Representatif (Mewakili Rakyat): * *
> 
>  *  * *Menjadi Perantara Rakyat: * * DPR berfungsi sebagai perantara aspirasi rakyat kepada pemerintah. 
>  *  * *Menyampaikan Kritik dan Saran: * * DPR dapat menyampaikan kritik dan saran kepada pemerintah terkait kebijakan yang dianggap tidak sesuai dengan kepentingan rakyat.
>  *  * *Melakukan Pengawasan terhadap Pemerintah: * * DPR memiliki wewenang untuk mengawasi kinerja pemerintah dan meminta pertanggungjawaban atas kebijakan yang diambil.
> 
>  * *3. Kontrol (Mencegah Penyalahgunaan Kekuasaan): * *
> 
>  *  * *Menetapkan dan Mengadili Presiden: * * DPR memiliki wewenang untuk menetapkan dan mengadili Presiden jika melakukan pelanggaran hukum.
>  *  * *Mengajukan Impeachment: * * DPR dapat mengajukan impeachment terhadap Presiden jika dianggap melakukan pelanggaran berat.
>  *  * *Mengawasi Penggunaan Kekuasaan: * * DPR memiliki wewenang untuk mengawasi penggunaan kekuasaan oleh pemerintah dan mencegah penyalahgunaan kekuasaan.
> 
>  * *4. Fungsi Lainnya: * *
> 
>  *  * *Memilih dan Mengangkat Menteri: * * DPR bersama Presiden memilih dan mengangkat Menteri.
>  *  * *Menentukan Kebijakan Luar Negeri: * * DPR memiliki wewenang untuk menentukan kebijakan luar negeri bersama pemerintah.
>  *  * *Menetapkan Perjanjian Internasional: * * DPR memiliki wewenang untuk menetapkan perjanjian internasional bersama pemerintah.
> 
>  * *Singkatnya, DPR berperan sebagai lembaga legislatif, representatif, kontrol, dan memiliki fungsi lainnya yang penting dalam sistem pemerintahan di Indonesia.  DPR berfungsi untuk memastikan bahwa pemerintah menjalankan tugasnya dengan baik dan sesuai dengan kepentingan rakyat. * * 
